# One tile

## Utils

In [1]:
!pip install fiona
!pip install rasterio
!pip install tqdm

     |████████████████████████████████| 14.8MB 313kB/s 
     |████████████████████████████████| 18.3MB 259kB/s 


In [2]:
import rasterio 
import matplotlib.pyplot as plt
import numpy as np
import fiona
import pandas as pd
import torch
torch.backends.cudnn.benchmark = True
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
import datetime
from tqdm import tqdm
from torchvision.transforms import transforms
import os
import sys
import numpy as np
import rasterio
from rasterio.windows import Window
from rasterio.errors import RasterioError, RasterioIOError
import torch 
from torchvision import transforms
from torch.utils.data.dataset import IterableDataset

In [4]:
#Mount the drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Split data in to train and validation

Using NLCD as labels, train with both years at the same time

In [5]:
image_fns = []
label_fns = []
groups = []
with fiona.open("/content/drive/MyDrive/Data/DFC_2021/dfc2021_index.geojson") as f:
    for row in f:
        properties = row["properties"]
        image_fns.append(properties["naip-2013"])
        label_fns.append(properties["nlcd-2013"])
        groups.append(0)
        
        image_fns.append(properties["naip-2017"])
        label_fns.append(properties["nlcd-2016"])
        groups.append(1)

df = pd.DataFrame.from_dict({
    "image_fn": image_fns,
    "label_fn": label_fns,
    "group": groups
})

df.to_csv("/content/drive/MyDrive/Data/DFC_2021/training_set_naip_nlcd_both.csv", index=False)

Using NLCD as labels, train with a single year at a time

2013 NAIP, 2013 NLCD

In [6]:
image_fns = []
label_fns = []
groups = []
with fiona.open("/content/drive/MyDrive/Data/DFC_2021/dfc2021_index.geojson") as f:
    for row in f:
        properties = row["properties"]
        image_fns.append(properties["naip-2013"])
        label_fns.append(properties["nlcd-2013"])
        groups.append(0)

df = pd.DataFrame.from_dict({
    "image_fn": image_fns,
    "label_fn": label_fns,
    "group": groups
})

df.to_csv("/content/drive/MyDrive/Data/DFC_2021/training_set_naip_nlcd_2013.csv", index=False)

2017 NAIP, 2017 NLCD

In [7]:
image_fns = []
label_fns = []
groups = []
with fiona.open("/content/drive/MyDrive/Data/DFC_2021/dfc2021_index.geojson") as f:
    for row in f:
        properties = row["properties"]
        image_fns.append(properties["naip-2017"])
        label_fns.append(properties["nlcd-2016"])
        groups.append(1)

df = pd.DataFrame.from_dict({
    "image_fn": image_fns,
    "label_fn": label_fns,
    "group": groups
})

df.to_csv("/content/drive/MyDrive/Data/DFC_2021/training_set_naip_nlcd_2017.csv", index=False)

## Dataset Loader

TileDataset

In [8]:
import numpy as np

import rasterio
from rasterio.windows import Window
from rasterio.errors import RasterioIOError

import torch
from torch.utils.data.dataset import Dataset

class TileInferenceDataset(Dataset):
    
    def __init__(self, fn, chip_size, stride, transform=None, windowed_sampling=False, verbose=False):
        """A torch Dataset for sampling a grid of chips that covers an input tile. 
        
        If `chip_size` doesn't divide the height of the tile evenly (which is what is likely to happen) then we will sample an additional row of chips that are aligned to the bottom of the file.
        We do a similar operation if `chip_size` doesn't divide the width of the tile evenly -- by appending an additional column.
        
        Note: without a `transform` we will return chips in (height, width, channels) format in whatever the tile's dtype is.
        
        Args:
            fn: The path to the file to sample from (this can be anything that rasterio.open(...) knows how to read).
            chip_size: The size of chips to return (chips will be squares).
            stride: How much we move the sliding window to sample the next chip. If this is is less than `chip_size` then we will get overlapping windows, if it is > `chip_size` then some parts of the tile will not be sampled.
            transform: A torchvision Transform to apply on each chip.
            windowed_sample: If `True` we will use rasterio.windows.Window to sample chips without every loading the entire file into memory, else, we will load the entire tile up-front and index into it to sample chips.
            verbose: Flag to control printing stuff.
        """
        self.fn = fn
        self.chip_size = chip_size
        
        self.transform = transform
        self.transform = transforms.Compose([transforms.ToTensor()])
        self.windowed_sampling = windowed_sampling
        self.verbose = verbose
        
        with rasterio.open(self.fn) as f:
            height, width = f.height, f.width
            self.num_channels = f.count
            self.dtype = f.profile["dtype"]
            if not windowed_sampling: # if we aren't using windowed sampling, then go ahead and read in all of the data
                self.data = np.rollaxis(f.read(), 0, 3)
            
        self.chip_coordinates = [] # upper left coordinate (y,x), of each chip that this Dataset will return
        for y in list(range(0, height - self.chip_size, stride)) + [height - self.chip_size]:
            for x in list(range(0, width - self.chip_size, stride)) + [width - self.chip_size]:
                self.chip_coordinates.append((y,x))
        self.num_chips = len(self.chip_coordinates)

        if self.verbose:
            print("Constructed TileInferenceDataset -- we have %d by %d file with %d channels with a dtype of %s. We are sampling %d chips from it." % (
                height, width, self.num_channels, self.dtype, self.num_chips
            ))
            
    def __getitem__(self, idx):
        '''
        Returns:
            A tuple (chip, (y,x)): `chip` is the chip that we sampled from the larger tile. (y,x) are the indices of the upper left corner of the chip.
        '''
        y, x = self.chip_coordinates[idx]
        
        if self.windowed_sampling:
            try:
                with rasterio.Env():
                    with rasterio.open(self.fn) as f:
                        img = np.rollaxis(f.read(window=rasterio.windows.Window(x, y, self.chip_size, self.chip_size)), 0, 3)
                        pritn(img)
            except RasterioIOError as e: # NOTE(caleb): I put this here to catch weird errors that I was seeing occasionally when trying to read from COGS - I don't remember the details though
                print("Reading %d failed, returning 0's" % (idx))
                img = np.zeros((self.chip_size, self.chip_size, self.num_channels), dtype=np.uint8)
        else:
            img = self.data[y:y+self.chip_size, x:x+self.chip_size]


        if self.transform is not None:
            img = self.transform(img)
        
        return img, np.array((y,x))
        
    def __len__(self):
        return self.num_chips

In [9]:
#-------------------
    # Run on each line in the input
    #-------------------
input_fn = '/content/drive/MyDrive/Data/DFC_2021/training_set_naip_nlcd_2013.csv'

input_dataframe = pd.read_csv(input_fn,usecols = ['image_fn'], nrows=21 )
#input_dataframe = pd.read_csv(input_fn)
image_fns = input_dataframe["image_fn"].values
#groups = input_dataframe["group"].values


lst = []
for image_idx in range(len(image_fns)):
  tic = time.time()
  image_fn = image_fns[image_idx]
  group = groups[image_idx]
  lst.append(image_fn)
  print(lst)


  print("(%d/%d) Processing %s" % (image_idx, len(image_fns), image_fn), end=" ... ")

['https://dfc2021.blob.core.windows.net/competition-data/naip-2013/546_naip-2013.tif']
(0/21) Processing https://dfc2021.blob.core.windows.net/competition-data/naip-2013/546_naip-2013.tif ... ['https://dfc2021.blob.core.windows.net/competition-data/naip-2013/546_naip-2013.tif', 'https://dfc2021.blob.core.windows.net/competition-data/naip-2013/597_naip-2013.tif']
(1/21) Processing https://dfc2021.blob.core.windows.net/competition-data/naip-2013/597_naip-2013.tif ... ['https://dfc2021.blob.core.windows.net/competition-data/naip-2013/546_naip-2013.tif', 'https://dfc2021.blob.core.windows.net/competition-data/naip-2013/597_naip-2013.tif', 'https://dfc2021.blob.core.windows.net/competition-data/naip-2013/596_naip-2013.tif']
(2/21) Processing https://dfc2021.blob.core.windows.net/competition-data/naip-2013/596_naip-2013.tif ... ['https://dfc2021.blob.core.windows.net/competition-data/naip-2013/546_naip-2013.tif', 'https://dfc2021.blob.core.windows.net/competition-data/naip-2013/597_naip-2013

In [11]:
class StreamingGeospatialDataset(IterableDataset):
    
    def __init__(self, imagery_fns, label_fns=None, groups=None, chip_size=32, num_chips_per_tile=200, windowed_sampling=False, image_transform=None, label_transform=None, nodata_check=None, verbose=False):
        """A torch Dataset for randomly sampling chips from a list of tiles. When used in conjunction with a DataLoader that has `num_workers>1` this Dataset will assign each worker to sample chips from disjoint sets of tiles.
        Args:
            imagery_fns: A list of filenames (or URLS -- anything that `rasterio.open()` can read) pointing to imagery tiles.
            label_fns: A list of filenames of the same size as `imagery_fns` pointing to label mask tiles or `None` if the Dataset should operate in "imagery only mode". Note that we expect `imagery_fns[i]` and `label_fns[i]` to have the same dimension and coordinate system.
            groups: Optional: A list of integers of the same size as `imagery_fns` that gives the "group" membership of each tile. This can be used to normalize imagery from different groups differently.
            chip_size: Desired size of chips (in pixels).
            num_chips_per_tile: Desired number of chips to sample for each tile.
            windowed_sampling: Flag indicating whether we should sample each chip with a read using `rasterio.windows.Window` or whether we should read the whole tile into memory, then sample chips.
            image_transform: A function to apply to each image chip object. If this is `None`, then the only transformation applied to the loaded imagery will be to convert it to a `torch.Tensor`. If this is not `None`, then the function should return a `Torch.tensor`. Further, if `groups` is not `None` then the transform function should expect the imagery as the first argument and the group as the second argument.
            label_transform: Similar to image_transform, but applied to label chips.
            nodata_check: A method that will check an `(image_chip)` or `(image_chip, label_chip)` (if `label_fns` are provided) and return whether or not the chip should be skipped. This can be used, for example, to skip chips that contain nodata values.
            verbose: If `False` we will be quiet.
        """

        if label_fns is None:
            self.fns = imagery_fns
            self.use_labels = False
        else:
            self.fns = list(zip(imagery_fns, label_fns)) 
            self.use_labels = True

        self.groups = groups

        self.chip_size = chip_size
        self.num_chips_per_tile = num_chips_per_tile
        self.windowed_sampling = windowed_sampling

        self.image_transform = image_transform
        self.label_transform = label_transform
        self.nodata_check = nodata_check

        self.verbose = verbose

        if self.verbose:
            print("Constructed StreamingGeospatialDataset")

    def stream_tile_fns(self):
        worker_info = torch.utils.data.get_worker_info()
        if worker_info is None: # In this case we are not loading through a DataLoader with multiple workers
            worker_id = 0
            num_workers = 1
        else:
            worker_id = worker_info.id
            num_workers = worker_info.num_workers

        # We only want to shuffle the order we traverse the files if we are the first worker (else, every worker will shuffle the files...)
        if worker_id == 0:
            np.random.shuffle(self.fns) # in place
        # NOTE: A warning, when different workers are created they will all have the same numpy random seed, however will have different torch random seeds. If you want to use numpy random functions, seed appropriately.
        #seed = torch.randint(low=0,high=2**32-1,size=(1,)).item()
        #np.random.seed(seed) # when different workers spawn, they have the same numpy random seed...

        if self.verbose:
            print("Creating a filename stream for worker %d" % (worker_id))

        # This logic splits up the list of filenames into `num_workers` chunks. Each worker will recieve ceil(num_filenames / num_workers) filenames to generate chips from. If the number of workers doesn't divide the number of filenames evenly then the last worker will have fewer filenames.
        N = len(self.fns)
        num_files_per_worker = int(np.ceil(N / num_workers))
        lower_idx = worker_id * num_files_per_worker
        upper_idx = min(N, (worker_id+1) * num_files_per_worker)
        for idx in range(lower_idx, upper_idx):

            label_fn = None
            if self.use_labels:
                img_fn, label_fn = self.fns[idx]
            else:
                img_fn = self.fns[idx]

            if self.groups is not None:
                group = self.groups[idx]
            else:
                group = None

            if self.verbose:
                print("Worker %d, yielding file %d" % (worker_id, idx))

            yield (img_fn, label_fn, group)

    def stream_chips(self):
        for img_fn, label_fn, group in self.stream_tile_fns():
            num_skipped_chips = 0

            # Open file pointers
            img_fp = rasterio.open(img_fn, "r")
            label_fp = rasterio.open(label_fn, "r") if self.use_labels else None

            height, width = img_fp.shape
            if self.use_labels: # garuntee that our label mask has the same dimensions as our imagery
                t_height, t_width = label_fp.shape
                assert height == t_height and width == t_width


            # If we aren't in windowed sampling mode then we should read the entire tile up front
            img_data = None
            label_data = None
            try:
                if not self.windowed_sampling:
                    img_data = np.rollaxis(img_fp.read(), 0, 3)
                    if self.use_labels:
                        label_data = label_fp.read().squeeze() # assume the label geotiff has a single channel
            except RasterioError as e:
                print("WARNING: Error reading in entire file, skipping to the next file")
                continue

            for i in range(self.num_chips_per_tile):
                # Select the top left pixel of our chip randomly
                x = np.random.randint(0, width-self.chip_size)
                y = np.random.randint(0, height-self.chip_size)

                # Read imagery / labels
                img = None
                labels = None
                if self.windowed_sampling:
                    try:
                        img = np.rollaxis(img_fp.read(window=Window(x, y, self.chip_size, self.chip_size)), 0, 3)
                        print(img.shape)
                        if self.use_labels:
                            labels = label_fp.read(window=Window(x, y, self.chip_size, self.chip_size)).squeeze()
                    except RasterioError:
                        print("WARNING: Error reading chip from file, skipping to the next chip")
                        continue
                else:
                    img = img_data[y:y+self.chip_size, x:x+self.chip_size, :]
                    if self.use_labels:
                        labels = label_data[y:y+self.chip_size, x:x+self.chip_size]

                # Check for no data
                if self.nodata_check is not None:
                    if self.use_labels:
                        skip_chip = self.nodata_check(img, labels)
                    else:
                        skip_chip = self.nodata_check(img)

                    if skip_chip: # The current chip has been identified as invalid by the `nodata_check(...)` method
                        num_skipped_chips += 1
                        continue

                # Transform the imagery
                if self.image_transform is not None:
                    if self.groups is None:
                        img = self.image_transform(img)
                    else:
                        img = self.image_transform(img, group)
                else:
                    img = torch.from_numpy(img).squeeze()

                # Transform the labels
                if self.use_labels:
                    if self.label_transform is not None:
                        if self.groups is None:
                            labels = self.label_transform(labels)
                        else:
                            labels = self.label_transform(labels, group)
                    else:
                        labels = torch.from_numpy(labels).squeeze()


                # Note, that img should be a torch "Double" type (i.e. a np.float32) and labels should be a torch "Long" type (i.e. np.int64)
                if self.use_labels:
                    yield img, labels
                else:
                    yield img

            # Close file pointers
            img_fp.close()
            if self.use_labels:
                label_fp.close()

            if num_skipped_chips>0 and self.verbose:
                print("We skipped %d chips on %s" % (img_fn))

    def __iter__(self):
        if self.verbose:
            print("Creating a new StreamingGeospatialDataset iterator")
        return iter(self.stream_chips())

Creating color map

In [12]:
input_fn = '/content/drive/MyDrive/Data/DFC_2021/training_set_naip_nlcd_2013.csv'
batch_size = 32

input_dataframe = pd.read_csv(input_fn, nrows=21 )
#input_dataframe = pd.read_csv(input_fn)
image_fns = input_dataframe["image_fn"].values
print(image_fns)
label_fns = input_dataframe["label_fn"].values
print(label_fns)
groups = input_dataframe["group"].values


['https://dfc2021.blob.core.windows.net/competition-data/naip-2013/546_naip-2013.tif'
 'https://dfc2021.blob.core.windows.net/competition-data/naip-2013/597_naip-2013.tif'
 'https://dfc2021.blob.core.windows.net/competition-data/naip-2013/596_naip-2013.tif'
 'https://dfc2021.blob.core.windows.net/competition-data/naip-2013/595_naip-2013.tif'
 'https://dfc2021.blob.core.windows.net/competition-data/naip-2013/541_naip-2013.tif'
 'https://dfc2021.blob.core.windows.net/competition-data/naip-2013/497_naip-2013.tif'
 'https://dfc2021.blob.core.windows.net/competition-data/naip-2013/496_naip-2013.tif'
 'https://dfc2021.blob.core.windows.net/competition-data/naip-2013/545_naip-2013.tif'
 'https://dfc2021.blob.core.windows.net/competition-data/naip-2013/544_naip-2013.tif'
 'https://dfc2021.blob.core.windows.net/competition-data/naip-2013/543_naip-2013.tif'
 'https://dfc2021.blob.core.windows.net/competition-data/naip-2013/542_naip-2013.tif'
 'https://dfc2021.blob.core.windows.net/competition-da

Hyper-parameters

In [19]:
NUM_WORKERS = 4
CHIP_SIZE = 32
PADDING = 28
assert PADDING % 2 == 0
HALF_PADDING = PADDING//2
CHIP_STRIDE = CHIP_SIZE - PADDING
batch_size = 32
CHIP_SIZE = 32
NUM_CHIPS_PER_TILE = 100

Transform Data

In [20]:
def image_transforms(img, group):
    if group == 0:
        img = (img - NAIP_2013_MEANS) / NAIP_2013_STDS
    elif group == 1:
        img = (img - NAIP_2017_MEANS) / NAIP_2017_STDS
    else:
        raise ValueError("group not recognized")
    img = np.rollaxis(img, 2, 0).astype(np.float32)
    img = torch.from_numpy(img)
    return img

def label_transforms(labels, group):
    labels = NLCD_CLASS_TO_IDX_MAP[labels]
    labels = torch.from_numpy(labels)
    return labels

def nodata_check(img, labels):
    return np.any(labels == 0) or np.any(np.sum(img == 0, axis=2) == 4)

DataLoader

In [21]:
with rasterio.open(image_fn) as f:
  img = f.read(1)
  input_width, input_height = f.width, f.height
  input_profile = f.profile.copy()
dataset = StreamingGeospatialDataset(
        imagery_fns= lst, label_fns =label_fns, groups=groups, chip_size=CHIP_SIZE, num_chips_per_tile=NUM_CHIPS_PER_TILE, windowed_sampling=False, verbose=False,
        image_transform=image_transforms, label_transform=label_transforms, nodata_check=nodata_check
    )
dataloader = torch.utils.data.DataLoader(
      dataset,batch_size=batch_size,
      num_workers=NUM_WORKERS,
      pin_memory=True,
        )


Check the size of the dataset

In [22]:
dataiter = iter(dataloader)
images, labels = dataiter.next()
print(images.shape)
print(labels.shape)

NameError: ignored

Copying the propertieds of the image

In [23]:
with rasterio.open(image_fn) as f:
  print(f.shape)
  input_width, input_height = f.width, f.height
  input_profile = f.profile.copy()

(3880, 3880)


## FCN Model

In [24]:
import functools
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

class FCN(nn.Module):

    def __init__(self, num_input_channels, num_output_classes, num_filters=64):
        super(FCN,self).__init__()

        self.conv1 = nn.Conv2d(num_input_channels, num_filters, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(num_filters, num_filters,        kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(num_filters, num_filters,        kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(num_filters, num_filters,        kernel_size=3, stride=1, padding=1)
        self.conv5 = nn.Conv2d(num_filters, num_filters,        kernel_size=3, stride=1, padding=1)
        self.last =  nn.Conv2d(num_filters, num_output_classes, kernel_size=1, stride=1, padding=0)

    def forward(self,inputs):
        x = F.relu(self.conv1(inputs))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        x = self.last(x)
        return x

In [25]:
model = FCN(num_input_channels=4, num_output_classes=len(NLCD_CLASSES))
print(model)

NameError: ignored

## Model training

In [ ]:
model = model
optimizer = optim.AdamW(model.parameters(), lr=0.00001, amsgrad=True)
criterion = nn.CrossEntropyLoss()
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, "min")
epochs = 50
num_batches= 32

In [ ]:
epoch = epochs
for epoch in range(epoch):
    losses = []
    for batch_idx, (data, targets) in tqdm(enumerate(dataloader), total=num_batches, file=sys.stdout):
    
        #optimizer.zero_grad()
        # get the inputs; data is a list of [inputs, labels]

        output = model(data)
        #print(output)
        #print(targets)
        print('the shape of output is',(output.shape))
        print('the shape of x is',data.shape)
        print('the shape of y is',targets.shape)
        #loss = criterion(output, torch.max(y, 1)[1])
        loss = criterion(output,targets)
        losses.append(loss.item())
        # zero the parameter gradients
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


        print("Loss at {}th epoch: {}".format(epoch,np.mean(losses)))

Streaming output truncated to the last 5000 lines.
Loss at 31th epoch: 0.8401385311569486
 88%|████████▊ | 28/32 [00:34<00:03,  1.26it/s]the shape of output is torch.Size([32, 17, 32, 32])
the shape of x is torch.Size([32, 4, 32, 32])
the shape of y is torch.Size([32, 32, 32])
Loss at 31th epoch: 0.8692714613059471
 91%|█████████ | 29/32 [00:34<00:02,  1.31it/s]the shape of output is torch.Size([32, 17, 32, 32])
the shape of x is torch.Size([32, 4, 32, 32])
the shape of y is torch.Size([32, 32, 32])
Loss at 31th epoch: 0.8866002261638641
 94%|█████████▍| 30/32 [00:35<00:01,  1.30it/s]the shape of output is torch.Size([32, 17, 32, 32])
the shape of x is torch.Size([32, 4, 32, 32])
the shape of y is torch.Size([32, 32, 32])
Loss at 31th epoch: 0.8764490985101269
 97%|█████████▋| 31/32 [00:36<00:00,  1.23it/s]the shape of output is torch.Size([32, 17, 32, 32])
the shape of x is torch.Size([32, 4, 32, 32])
the shape of y is torch.Size([32, 32, 32])
Loss at 31th epoch: 0.8676420357078314
10

In [ ]:
output.shape

torch.Size([23, 17, 32, 32])

In [ ]:
output = output / output.sum(axis=0, keepdims=True)
output = (output * 255).astype(np.uint8)

output_profile = input_profile.copy()
  
output_profile["driver"] = "GTiff"
output_profile["dtype"] = "uint8"
output_profile["count"] = len(NLCD_CLASSES)
del output_profile["nodata"]

output_fn = image_fn.split("/")[-1] # something like "546_naip-2013.tif"
output_fn = output_fn.replace("naip", "predictions-soft")
output_fn = os.path.join(output_dir, output_fn)

with rasterio.open(output_fn, "w", **output_profile) as f:
  print(f.shape)
  f.write(output.astype(rasterio.uint8))

In [ ]:
output_dir = '/content/drive/MyDrive/Data/DFC_2021'

output_profile = input_profile.copy()
output_profile["driver"] = "GTiff"
output_profile["dtype"] = "uint8"
output_profile["count"] = 1
output_profile["nodata"] = 0

output_fn = image_fn.split("/")[-1] # something like "546_naip-2013.tif"
output_fn = output_fn.replace("naip", "predictions")
output_fn = os.path.join(output_dir, output_fn)

with rasterio.open(output_fn, "w", **output_profile) as f:
  f.write(output_hard, 1)
  f.write_colormap(1,NLCD_IDX_COLORMAP)

In [ ]:
pred = output.detach().numpy()
#np.save(pred, '/content/drive/MyDrive/Data/DFC_2021/predictions.npy')

In [ ]:
output_dir = '/content/drive/MyDrive/Data/DFC_2021'

output_profile = input_profile.copy()
output_profile["driver"] = "GTiff"
output_profile["dtype"] = "uint8"
output_profile["count"] = 1
output_profile["nodata"] = 0

output_fn = image_fn.split("/")[-1] # something like "546_naip-2013.tif"
output_fn = output_fn.replace("naip", "predictions")
output_fn = os.path.join(output_dir, output_fn)

with rasterio.open(output_fn, "w", **output_profile) as f:
  f.write(output_hard, 1)
  f.write_colormap(1,NLCD_IDX_COLORMAP)


dtype('float32')

In [ ]:
prediction = output[1,:,:,:]
prediction = output.detach().numpy()
prediction.shape
print(prediction.dtype)
prediction = prediction / prediction.sum(axis=0, keepdims=True)
prediction = (prediction * 255).astype(np.uint8)
prediction.shape

float32


(23, 17, 32, 32)

In [ ]:
prediction = prediction[1,:,:,:]
prediction = prediction / prediction.sum(axis=0, keepdims=True)
prediction = (prediction * 255).astype(np.uint8)
prediction.shape


TypeError: ignored

In [ ]:
output_dir = '/content/drive/MyDrive/Data/DFC_2021'

output_profile = input_profile.copy()
output_profile["driver"] = "GTiff"
output_profile["dtype"] = "uint8"
output_profile["count"] = 1
output_profile["nodata"] = 0

output_fn = image_fn.split("/")[-1] # something like "546_naip-2013.tif"
output_fn = output_fn.replace("naip", "predictions")
output_fn = os.path.join(output_dir, output_fn)

with rasterio.open(output_fn, "w", **output_profile) as f:
  f.write(output_hard, 1)
  f.write_colormap(1,NLCD_IDX_COLORMAP)



'\n#if args.save_soft:\n  output = output / output.sum(axis=0, keepdims=True)\n  output = (output * 255).astype(np.uint8)\n\n  output_profile = input_profile.copy()\n  output_profile["driver"] = "GTiff"\n  output_profile["dtype"] = "uint8"\n  output_profile["count"] = len(NLCD_CLASSES)\n  del output_profile["nodata"]\n\n  output_fn = image_fn.split("/")[-1] # something like "546_naip-2013.tif"\n  output_fn = output_fn.replace("naip", "predictions-soft")\n # output_fn = os.path.join(output_dir, output_fn)\n\n  with rasterio.open(output_fn, "w", **output_profile) as f:\n    print(f.shape)\n    f.write(output) '

In [ ]:

output = output / output.sum(axis=0, keepdims=True)
output = (output * 255).astype(np.uint8)

output_profile = input_profile.copy()
  
output_profile["driver"] = "GTiff"
output_profile["dtype"] = "uint8"
output_profile["count"] = len(NLCD_CLASSES)
del output_profile["nodata"]

output_fn = image_fn.split("/")[-1] # something like "546_naip-2013.tif"
output_fn = output_fn.replace("naip", "predictions-soft")
output_fn = os.path.join(output_dir, output_fn)

with rasterio.open(output_fn, "w", **output_profile) as f:
  print(f.shape)
  f.write(output.astype(rasterio.uint8))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in true_divide
  


(3880, 3880)


# Streamline tiles

In [ ]:


NAIP_2013_MEANS = np.array([117.00, 130.75, 122.50, 159.30])
NAIP_2013_STDS = np.array([38.16, 36.68, 24.30, 66.22])
NAIP_2017_MEANS = np.array([72.84,  86.83, 76.78, 130.82])
NAIP_2017_STDS = np.array([41.78, 34.66, 28.76, 58.95])
NLCD_CLASSES = [ 0, 11, 12, 21, 22, 23, 24, 31, 41, 42, 43, 52, 71, 81, 82, 90, 95] # 16 classes + 1 nodata class ("0"). Note that "12" is "Perennial Ice/Snow" and is not present in Maryland.

NLCD_CLASS_COLORMAP = { # Copied from the emebedded color table in the NLCD data files
    0:  (0, 0, 0, 255),
    11: (70, 107, 159, 255),
    12: (209, 222, 248, 255),
    21: (222, 197, 197, 255),
    22: (217, 146, 130, 255),
    23: (235, 0, 0, 255),
    24: (171, 0, 0, 255),
    31: (179, 172, 159, 255),
    41: (104, 171, 95, 255),
    42: (28, 95, 44, 255),
    43: (181, 197, 143, 255),
    52: (204, 184, 121, 255),
    71: (223, 223, 194, 255),
    81: (220, 217, 57, 255),
    82: (171, 108, 40, 255),
    90: (184, 217, 235, 255),
    95: (108, 159, 184, 255)
}

NLCD_IDX_COLORMAP = {
    idx: NLCD_CLASS_COLORMAP[c]
    for idx, c in enumerate(NLCD_CLASSES)
}

def get_nlcd_class_to_idx_map():
    nlcd_label_to_idx_map = []
    idx = 0
    for i in range(NLCD_CLASSES[-1]+1):
        if i in NLCD_CLASSES:
            nlcd_label_to_idx_map.append(idx)
            idx += 1
        else:
            nlcd_label_to_idx_map.append(0)
    nlcd_label_to_idx_map = np.array(nlcd_label_to_idx_map).astype(np.int64)
    return nlcd_label_to_idx_map

NLCD_CLASS_TO_IDX_MAP = get_nlcd_class_to_idx_map() # I do this computation on import for illustration (this could instead be a length 96 vector that is hardcoded here)


NLCD_IDX_TO_REDUCED_LC_MAP = np.array([
    4,#  0 No data 0
    0,#  1 Open Water
    4,#  2 Ice/Snow
    2,#  3 Developed Open Space
    3,#  4 Developed Low Intensity
    3,#  5 Developed Medium Intensity
    3,#  6 Developed High Intensity
    3,#  7 Barren Land
    1,#  8 Deciduous Forest
    1,#  9 Evergreen Forest
    1,# 10 Mixed Forest
    1,# 11 Shrub/Scrub
    2,# 12 Grassland/Herbaceous
    2,# 13 Pasture/Hay
    2,# 14 Cultivated Crops
    1,# 15 Woody Wetlands
    1,# 16 Emergent Herbaceious Wetlands
])

NLCD_IDX_TO_REDUCED_LC_ACCUMULATOR = np.array([
    [0,0,0,0,1],#  0 No data 0
    [1,0,0,0,0],#  1 Open Water
    [0,0,0,0,1],#  2 Ice/Snow
    [0,0,0,0,0],#  3 Developed Open Space
    [0,0,0,0,0],#  4 Developed Low Intensity
    [0,0,0,1,0],#  5 Developed Medium Intensity
    [0,0,0,1,0],#  6 Developed High Intensity
    [0,0,0,0,0],#  7 Barren Land
    [0,1,0,0,0],#  8 Deciduous Forest
    [0,1,0,0,0],#  9 Evergreen Forest
    [0,1,0,0,0],# 10 Mixed Forest
    [0,1,0,0,0],# 11 Shrub/Scrub
    [0,0,1,0,0],# 12 Grassland/Herbaceous
    [0,0,1,0,0],# 13 Pasture/Hay
    [0,0,1,0,0],# 14 Cultivated Crops
    [0,1,0,0,0],# 15 Woody Wetlands
    [0,1,0,0,0],# 16 Emergent Herbaceious Wetlands
])



## Streamline DataLoader

In [ ]:


class StreamingGeospatialDataset(IterableDataset):
    
    def __init__(self, imagery_fns, label_fns=None, groups=None, chip_size=256, num_chips_per_tile=200, windowed_sampling=False, image_transform=None, label_transform=None, nodata_check=None, verbose=False):
        """A torch Dataset for randomly sampling chips from a list of tiles. When used in conjunction with a DataLoader that has `num_workers>1` this Dataset will assign each worker to sample chips from disjoint sets of tiles.
        Args:
            imagery_fns: A list of filenames (or URLS -- anything that `rasterio.open()` can read) pointing to imagery tiles.
            label_fns: A list of filenames of the same size as `imagery_fns` pointing to label mask tiles or `None` if the Dataset should operate in "imagery only mode". Note that we expect `imagery_fns[i]` and `label_fns[i]` to have the same dimension and coordinate system.
            groups: Optional: A list of integers of the same size as `imagery_fns` that gives the "group" membership of each tile. This can be used to normalize imagery from different groups differently.
            chip_size: Desired size of chips (in pixels).
            num_chips_per_tile: Desired number of chips to sample for each tile.
            windowed_sampling: Flag indicating whether we should sample each chip with a read using `rasterio.windows.Window` or whether we should read the whole tile into memory, then sample chips.
            image_transform: A function to apply to each image chip object. If this is `None`, then the only transformation applied to the loaded imagery will be to convert it to a `torch.Tensor`. If this is not `None`, then the function should return a `Torch.tensor`. Further, if `groups` is not `None` then the transform function should expect the imagery as the first argument and the group as the second argument.
            label_transform: Similar to image_transform, but applied to label chips.
            nodata_check: A method that will check an `(image_chip)` or `(image_chip, label_chip)` (if `label_fns` are provided) and return whether or not the chip should be skipped. This can be used, for example, to skip chips that contain nodata values.
            verbose: If `False` we will be quiet.
        """

        if label_fns is None:
            self.fns = imagery_fns
            self.use_labels = False
        else:
            self.fns = list(zip(imagery_fns, label_fns)) 
            self.use_labels = True

        self.groups = groups

        self.chip_size = chip_size
        self.num_chips_per_tile = num_chips_per_tile
        self.windowed_sampling = windowed_sampling

        self.image_transform = image_transform
        self.label_transform = label_transform
        self.nodata_check = nodata_check

        self.verbose = verbose

        if self.verbose:
            print("Constructed StreamingGeospatialDataset")

    def stream_tile_fns(self):
        worker_info = torch.utils.data.get_worker_info()
        if worker_info is None: # In this case we are not loading through a DataLoader with multiple workers
            worker_id = 0
            num_workers = 1
        else:
            worker_id = worker_info.id
            num_workers = worker_info.num_workers

        # We only want to shuffle the order we traverse the files if we are the first worker (else, every worker will shuffle the files...)
        if worker_id == 0:
            np.random.shuffle(self.fns) # in place
        # NOTE: A warning, when different workers are created they will all have the same numpy random seed, however will have different torch random seeds. If you want to use numpy random functions, seed appropriately.
        #seed = torch.randint(low=0,high=2**32-1,size=(1,)).item()
        #np.random.seed(seed) # when different workers spawn, they have the same numpy random seed...

        if self.verbose:
            print("Creating a filename stream for worker %d" % (worker_id))

        # This logic splits up the list of filenames into `num_workers` chunks. Each worker will recieve ceil(num_filenames / num_workers) filenames to generate chips from. If the number of workers doesn't divide the number of filenames evenly then the last worker will have fewer filenames.
        N = len(self.fns)
        num_files_per_worker = int(np.ceil(N / num_workers))
        lower_idx = worker_id * num_files_per_worker
        upper_idx = min(N, (worker_id+1) * num_files_per_worker)
        for idx in range(lower_idx, upper_idx):

            label_fn = None
            if self.use_labels:
                img_fn, label_fn = self.fns[idx]
            else:
                img_fn = self.fns[idx]

            if self.groups is not None:
                group = self.groups[idx]
            else:
                group = None

            if self.verbose:
                print("Worker %d, yielding file %d" % (worker_id, idx))

            yield (img_fn, label_fn, group)

    def stream_chips(self):
        for img_fn, label_fn, group in self.stream_tile_fns():
            num_skipped_chips = 0

            # Open file pointers
            img_fp = rasterio.open(img_fn, "r")
            label_fp = rasterio.open(label_fn, "r") if self.use_labels else None

            height, width = img_fp.shape
            if self.use_labels: # garuntee that our label mask has the same dimensions as our imagery
                t_height, t_width = label_fp.shape
                assert height == t_height and width == t_width


            # If we aren't in windowed sampling mode then we should read the entire tile up front
            img_data = None
            label_data = None
            try:
                if not self.windowed_sampling:
                    img_data = np.rollaxis(img_fp.read(), 0, 3)
                    if self.use_labels:
                        label_data = label_fp.read().squeeze() # assume the label geotiff has a single channel
            except RasterioError as e:
                print("WARNING: Error reading in entire file, skipping to the next file")
                continue

            for i in range(self.num_chips_per_tile):
                # Select the top left pixel of our chip randomly
                x = np.random.randint(0, width-self.chip_size)
                y = np.random.randint(0, height-self.chip_size)

                # Read imagery / labels
                img = None
                labels = None
                if self.windowed_sampling:
                    try:
                        img = np.rollaxis(img_fp.read(window=Window(x, y, self.chip_size, self.chip_size)), 0, 3)
                        print(img.shape)
                        if self.use_labels:
                            labels = label_fp.read(window=Window(x, y, self.chip_size, self.chip_size)).squeeze()
                    except RasterioError:
                        print("WARNING: Error reading chip from file, skipping to the next chip")
                        continue
                else:
                    img = img_data[y:y+self.chip_size, x:x+self.chip_size, :]
                    if self.use_labels:
                        labels = label_data[y:y+self.chip_size, x:x+self.chip_size]

                # Check for no data
                if self.nodata_check is not None:
                    if self.use_labels:
                        skip_chip = self.nodata_check(img, labels)
                    else:
                        skip_chip = self.nodata_check(img)

                    if skip_chip: # The current chip has been identified as invalid by the `nodata_check(...)` method
                        num_skipped_chips += 1
                        continue

                # Transform the imagery
                if self.image_transform is not None:
                    if self.groups is None:
                        img = self.image_transform(img)
                    else:
                        img = self.image_transform(img, group)
                else:
                    img = torch.from_numpy(img).squeeze()

                # Transform the labels
                if self.use_labels:
                    if self.label_transform is not None:
                        if self.groups is None:
                            labels = self.label_transform(labels)
                        else:
                            labels = self.label_transform(labels, group)
                    else:
                        labels = torch.from_numpy(labels).squeeze()


                # Note, that img should be a torch "Double" type (i.e. a np.float32) and labels should be a torch "Long" type (i.e. np.int64)
                if self.use_labels:
                    yield img, labels
                else:
                    yield img

            # Close file pointers
            img_fp.close()
            if self.use_labels:
                label_fp.close()

            if num_skipped_chips>0 and self.verbose:
                print("We skipped %d chips on %s" % (img_fn))

    def __iter__(self):
        if self.verbose:
            print("Creating a new StreamingGeospatialDataset iterator")
        return iter(self.stream_chips())

In [ ]:
NUM_WORKERS = 0
NUM_CHIPS_PER_TILE = 100
CHIP_SIZE = 32

NAIP_2013_MEANS = np.array([117.00, 130.75, 122.50, 159.30])
NAIP_2013_STDS = np.array([38.16, 36.68, 24.30, 66.22])
NAIP_2017_MEANS = np.array([72.84,  86.83, 76.78, 130.82])
NAIP_2017_STDS = np.array([41.78, 34.66, 28.76, 58.95])
NLCD_CLASSES = [ 0, 11, 12, 21, 22, 23, 24, 31, 41, 42, 43, 52, 71, 81, 82, 90, 95] # 16 classes + 1 nodata class ("0"). Note that "12" is "Perennial Ice/Snow" and is not present in Maryland.

NLCD_CLASS_COLORMAP = { # Copied from the emebedded color table in the NLCD data files
    0:  (0, 0, 0, 255),
    11: (70, 107, 159, 255),
    12: (209, 222, 248, 255),
    21: (222, 197, 197, 255),
    22: (217, 146, 130, 255),
    23: (235, 0, 0, 255),
    24: (171, 0, 0, 255),
    31: (179, 172, 159, 255),
    41: (104, 171, 95, 255),
    42: (28, 95, 44, 255),
    43: (181, 197, 143, 255),
    52: (204, 184, 121, 255),
    71: (223, 223, 194, 255),
    81: (220, 217, 57, 255),
    82: (171, 108, 40, 255),
    90: (184, 217, 235, 255),
    95: (108, 159, 184, 255)
}

NLCD_IDX_COLORMAP = {
    idx: NLCD_CLASS_COLORMAP[c]
    for idx, c in enumerate(NLCD_CLASSES)
}

def get_nlcd_class_to_idx_map():
    nlcd_label_to_idx_map = []
    idx = 0
    for i in range(NLCD_CLASSES[-1]+1):
        if i in NLCD_CLASSES:
            nlcd_label_to_idx_map.append(idx)
            idx += 1
        else:
            nlcd_label_to_idx_map.append(0)
    nlcd_label_to_idx_map = np.array(nlcd_label_to_idx_map).astype(np.int64)
    return nlcd_label_to_idx_map

NLCD_CLASS_TO_IDX_MAP = get_nlcd_class_to_idx_map() # I do this computation on import for illustration (this could instead be a length 96 vector that is hardcoded here)


NLCD_IDX_TO_REDUCED_LC_MAP = np.array([
    4,#  0 No data 0
    0,#  1 Open Water
    4,#  2 Ice/Snow
    2,#  3 Developed Open Space
    3,#  4 Developed Low Intensity
    3,#  5 Developed Medium Intensity
    3,#  6 Developed High Intensity
    3,#  7 Barren Land
    1,#  8 Deciduous Forest
    1,#  9 Evergreen Forest
    1,# 10 Mixed Forest
    1,# 11 Shrub/Scrub
    2,# 12 Grassland/Herbaceous
    2,# 13 Pasture/Hay
    2,# 14 Cultivated Crops
    1,# 15 Woody Wetlands
    1,# 16 Emergent Herbaceious Wetlands
])

NLCD_IDX_TO_REDUCED_LC_ACCUMULATOR = np.array([
    [0,0,0,0,1],#  0 No data 0
    [1,0,0,0,0],#  1 Open Water
    [0,0,0,0,1],#  2 Ice/Snow
    [0,0,0,0,0],#  3 Developed Open Space
    [0,0,0,0,0],#  4 Developed Low Intensity
    [0,0,0,1,0],#  5 Developed Medium Intensity
    [0,0,0,1,0],#  6 Developed High Intensity
    [0,0,0,0,0],#  7 Barren Land
    [0,1,0,0,0],#  8 Deciduous Forest
    [0,1,0,0,0],#  9 Evergreen Forest
    [0,1,0,0,0],# 10 Mixed Forest
    [0,1,0,0,0],# 11 Shrub/Scrub
    [0,0,1,0,0],# 12 Grassland/Herbaceous
    [0,0,1,0,0],# 13 Pasture/Hay
    [0,0,1,0,0],# 14 Cultivated Crops
    [0,1,0,0,0],# 15 Woody Wetlands
    [0,1,0,0,0],# 16 Emergent Herbaceious Wetlands
])


def image_transforms(img, group):
    if group == 0:
        img = (img - NAIP_2013_MEANS) / NAIP_2013_STDS
    elif group == 1:
        img = (img - NAIP_2017_MEANS) / NAIP_2017_STDS
    else:
        raise ValueError("group not recognized")
    img = np.rollaxis(img, 2, 0).astype(np.float32)
    img = torch.from_numpy(img)
    return img

def label_transforms(labels, group):
    labels = NLCD_CLASS_TO_IDX_MAP[labels]
    labels = torch.from_numpy(labels)
    return labels

def nodata_check(img, labels):
    return np.any(labels == 0) or np.any(np.sum(img == 0, axis=2) == 4)


def main():
    print("Starting DFC2021 baseline training script at %s" % (str(datetime.datetime.now())))


In [ ]:
device = torch.device("cuda:0")
device

device(type='cuda', index=0)

In [ ]:
#-------------------
# Load input data
#-------------------

input_fn = '/content/drive/MyDrive/Data/DFC_2021/training_set_naip_nlcd_2013.csv'
batch_size = 32


input_dataframe = pd.read_csv(input_fn)
image_fns = input_dataframe["image_fn"].values
print(image_fns)
label_fns = input_dataframe["label_fn"].values
groups = input_dataframe["group"].values

#dataset = StreamingGeospatialDataset(
#imagery_fns=image_fns, label_fns=label_fns, groups=groups, chip_size=CHIP_SIZE, num_chips_per_tile=NUM_CHIPS_PER_TILE, windowed_sampling=False, verbose=False,
#image_transform=image_transforms, label_transform=label_transforms, nodata_check=nodata_check
#    )

dataset = StreamingGeospatialDataset(
imagery_fns=image_fns, label_fns=label_fns, groups=groups, chip_size=CHIP_SIZE, num_chips_per_tile=NUM_CHIPS_PER_TILE, windowed_sampling=False, verbose=False,
image_transform=image_transforms, label_transform=label_transforms, nodata_check=nodata_check
    )

dataloader = torch.utils.data.DataLoader(
dataset,
batch_size= batch_size,
num_workers=NUM_WORKERS,
pin_memory=True,
    )
num_training_batches_per_epoch = int(len(image_fns) * NUM_CHIPS_PER_TILE / batch_size)
print("We will be training with %d batches per epoch" % (num_training_batches_per_epoch))

['https://dfc2021.blob.core.windows.net/competition-data/naip-2013/546_naip-2013.tif'
 'https://dfc2021.blob.core.windows.net/competition-data/naip-2013/597_naip-2013.tif'
 'https://dfc2021.blob.core.windows.net/competition-data/naip-2013/596_naip-2013.tif'
 ...
 'https://dfc2021.blob.core.windows.net/competition-data/naip-2013/549_naip-2013.tif'
 'https://dfc2021.blob.core.windows.net/competition-data/naip-2013/548_naip-2013.tif'
 'https://dfc2021.blob.core.windows.net/competition-data/naip-2013/547_naip-2013.tif']
We will be training with 7031 batches per epoch


In [ ]:
dataiter = iter(dataloader)
images, labels = dataiter.next()
print(images.shape)
print(labels.shape)

torch.Size([32, 4, 32, 32])
torch.Size([32, 32, 32])


## FCN Model

In [ ]:
import functools
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

class FCN(nn.Module):

    def __init__(self, num_input_channels, num_output_classes, num_filters=64):
        super(FCN,self).__init__()

        self.conv1 = nn.Conv2d(num_input_channels, num_filters, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(num_filters, num_filters,        kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(num_filters, num_filters,        kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(num_filters, num_filters,        kernel_size=3, stride=1, padding=1)
        self.conv5 = nn.Conv2d(num_filters, num_filters,        kernel_size=3, stride=1, padding=1)
        self.last =  nn.Conv2d(num_filters, num_output_classes, kernel_size=1, stride=1, padding=0)

    def forward(self,inputs):
        x = F.relu(self.conv1(inputs))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        x = self.last(x)
        return x

In [ ]:
model = FCN(num_input_channels=4, num_output_classes=len(NLCD_CLASSES))
print(model)

FCN(
  (conv1): Conv2d(4, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (last): Conv2d(64, 17, kernel_size=(1, 1), stride=(1, 1))
)


## Training

In [ ]:
model = model
optimizer = optim.AdamW(model.parameters(), lr=0.00001, amsgrad=True)
criterion = nn.CrossEntropyLoss()
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, "min")
epochs = 50
num_batches= 32
#print("Model has %d parameters" % (utils.count_parameters(model)))

In [ ]:
epoch = epochs
for epoch in range(epoch):
    losses = []
    for batch_idx, (data, targets) in tqdm(enumerate(dataloader), total=num_batches, file=sys.stdout):
    
        #optimizer.zero_grad()
        # get the inputs; data is a list of [inputs, labels]

        output = model(data)
        #print(output)
        #print(targets)
        print('the shape of output is',(output.shape))
        print('the shape of x is',data.shape)
        print('the shape of y is',targets.shape)
        #loss = criterion(output, torch.max(y, 1)[1])
        loss = criterion(output,targets)
        losses.append(loss.item())
        # zero the parameter gradients
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


        print("Loss at {}th epoch: {}".format(epoch,np.mean(losses)))






  0%|          | 0/32 [00:00<?, ?it/s]the shape of output is torch.Size([32, 17, 32, 32])
the shape of x is torch.Size([32, 4, 32, 32])
the shape of y is torch.Size([32, 32, 32])
Loss at 0th epoch: 2.1296398639678955





  3%|▎         | 1/32 [00:05<02:46,  5.39s/it]the shape of output is torch.Size([32, 17, 32, 32])
the shape of x is torch.Size([32, 4, 32, 32])
the shape of y is torch.Size([32, 32, 32])
Loss at 0th epoch: 2.049004375934601





  6%|▋         | 2/32 [00:06<01:58,  3.96s/it]the shape of output is torch.Size([32, 17, 32, 32])
the shape of x is torch.Size([32, 4, 32, 32])
the shape of y is torch.Size([32, 32, 32])
Loss at 0th epoch: 2.0114193757375083





  9%|▉         | 3/32 [00:06<01:25,  2.96s/it]the shape of output is torch.Size([32, 17, 32, 32])
the shape of x is torch.Size([32, 4, 32, 32])
the shape of y is torch.Size([32, 32, 32])
Loss at 0th epoch: 2.000472068786621





 12%|█▎        | 4/32 [00:10<01:31,  3.26s/it]the shape of output is torch.Size([32, 

In [ ]:
def fit (model,data_loader, num_batches, optimizer, criterion, epoch, memo=''):
    model.train()
    
    losses = []
    tic = time.time()
    for batch_idx, (data, targets) in tqdm(enumerate(data_loader), total=num_batches, file=sys.stdout):
        data = data
        targets = targets
        
        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, targets)
        losses.append(loss.item())
        loss.backward()
        optimizer.step()

    avg_loss = np.mean(losses)
    
    print('[{}] Training Epoch: {}\t Time elapsed: {:.2f} seconds\t Loss: {:.2f}'.format(
        memo, epoch, time.time()-tic, avg_loss), end=""
    )
    print("")
    
    return [avg_loss],output

In [ ]:
fit(model,data_loader=dataloader, num_batches = batch_size, optimizer = optimizer, criterion =criterion, epoch = epochs, memo='')

135it [04:56,  1.72s/it]

KeyboardInterrupt: ignored

In [ ]:
import sys
import os
os.environ["CURL_CA_BUNDLE"] = "/etc/ssl/certs/ca-certificates.crt" # A workaround in case this happens: https://github.com/mapbox/rasterio/issues/1289
import time
import datetime
import argparse
import copy

import numpy as np
import pandas as pd

#from dataloaders.StreamingDatasets import StreamingGeospatialDataset

import torch
torch.backends.cudnn.deterministic = False
torch.backends.cudnn.benchmark = True
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import models
import utils

NUM_WORKERS = 4
NUM_CHIPS_PER_TILE = 100
CHIP_SIZE = 256


def image_transforms(img, group):
    if group == 0:
        img = (img - utils.NAIP_2013_MEANS) / utils.NAIP_2013_STDS
    elif group == 1:
        img = (img - utils.NAIP_2017_MEANS) / utils.NAIP_2017_STDS
    else:
        raise ValueError("group not recognized")
    img = np.rollaxis(img, 2, 0).astype(np.float32)
    img = torch.from_numpy(img)
    return img

def label_transforms(labels, group):
    labels = utils.NLCD_CLASS_TO_IDX_MAP[labels]
    labels = torch.from_numpy(labels)
    return labels

def nodata_check(img, labels):
    return np.any(labels == 0) or np.any(np.sum(img == 0, axis=2) == 4)


def main():
    print("Starting DFC2021 baseline training script at %s" % (str(datetime.datetime.now())))


    #-------------------
    # Setup
    #-------------------
    assert os.path.exists(args.input_fn)

    if os.path.isfile(args.output_dir):
        print("A file was passed as `--output_dir`, please pass a directory!")
        return

    if os.path.exists(args.output_dir) and len(os.listdir(args.output_dir)):
        if args.overwrite:
            print("WARNING! The output directory, %s, already exists, we might overwrite data in it!" % (args.output_dir))
        else:
            print("The output directory, %s, already exists and isn't empty. We don't want to overwrite and existing results, exiting..." % (args.output_dir))
            return
    else:
        print("The output directory doesn't exist or is empty.")
        os.makedirs(args.output_dir, exist_ok=True)

    if torch.cuda.is_available():
        device = torch.device("cuda:%d" % args.gpu)
    else:
        print("WARNING! Torch is reporting that CUDA isn't available, exiting...")
        return

    np.random.seed(args.seed)
    torch.manual_seed(args.seed)


    #-------------------
    # Load input data
    #-------------------
    input_dataframe = pd.read_csv(input_fn)
    image_fns = input_dataframe["image_fn"].values
    label_fns = input_dataframe["label_fn"].values
    groups = input_dataframe["group"].values

    dataset = StreamingGeospatialDataset(
        imagery_fns=image_fns, label_fns=label_fns, groups=groups, chip_size=CHIP_SIZE, num_chips_per_tile=NUM_CHIPS_PER_TILE, windowed_sampling=False, verbose=False,
        image_transform=image_transforms, label_transform=label_transforms, nodata_check=nodata_check
    )

    dataloader = torch.utils.data.DataLoader(
        dataset,
        batch_size=args.batch_size,
        num_workers=NUM_WORKERS,
        pin_memory=True,
    )

    num_training_batches_per_epoch = int(len(image_fns) * NUM_CHIPS_PER_TILE /batch_size)
    print("We will be training with %d batches per epoch" % (num_training_batches_per_epoch))


    #-------------------
    # Setup training
    #-------------------
    if args.model == "unet":
        model = models.get_unet()
    elif args.model == "fcn":
        model = models.get_fcn()
    else:
        raise ValueError("Invalid model")

    model = model.to(device)
    optimizer = optim.AdamW(model.parameters(), lr=0.001, amsgrad=True)
    criterion = nn.CrossEntropyLoss()
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, "min")

    print("Model has %d parameters" % (utils.count_parameters(model)))


    #-------------------
    # Model training
    #-------------------
    training_task_losses = []
    num_times_lr_dropped = 0 
    model_checkpoints = []
    temp_model_fn = os.path.join(args.output_dir, "most_recent_model.pt")

    for epoch in range(args.num_epochs):
        lr = utils.get_lr(optimizer)

        training_losses = utils.fit(
            model,
            device,
            dataloader,
            num_training_batches_per_epoch,
            optimizer,
            criterion,
            epoch,
        )
        scheduler.step(training_losses[0])

        model_checkpoints.append(copy.deepcopy(model.state_dict()))
        if args.save_most_recent:
            torch.save(model.state_dict(), temp_model_fn)

        if utils.get_lr(optimizer) < lr:
            num_times_lr_dropped += 1
            print("")
            print("Learning rate dropped")
            print("")
            
        training_task_losses.append(training_losses[0])
            
        if num_times_lr_dropped == 4:
            break


    #-------------------
    # Save everything
    #-------------------
    save_obj = {
        #'args': args,
        'training_task_losses': training_task_losses,
        "checkpoints": model_checkpoints
    }

    save_obj_fn = "results.pt"
    with open(os.path.join(args.output_dir, save_obj_fn), 'wb') as f:
        torch.save(save_obj, f)

if __name__ == "__main__":
    main()

ModuleNotFoundError: ignored

Model

In [ ]:
import functools

import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

import segmentation_models_pytorch as smp

import utils

class FCN(nn.Module):

    def __init__(self, num_input_channels, num_output_classes, num_filters=64):
        super(FCN,self).__init__()

        self.conv1 = nn.Conv2d(num_input_channels, num_filters, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(num_filters, num_filters,        kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(num_filters, num_filters,        kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(num_filters, num_filters,        kernel_size=3, stride=1, padding=1)
        self.conv5 = nn.Conv2d(num_filters, num_filters,        kernel_size=3, stride=1, padding=1)
        self.last =  nn.Conv2d(num_filters, num_output_classes, kernel_size=1, stride=1, padding=0)

    def forward(self,inputs):
        x = F.relu(self.conv1(inputs))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        x = self.last(x)
        return x


def get_unet():
    return smp.Unet(
        encoder_name='resnet18', encoder_depth=3, encoder_weights=None,
        decoder_channels=(128, 64, 64), in_channels=4, classes=len(utils.NLCD_CLASSES)
    )

def get_fcn():
    return FCN(num_input_channels=4, num_output_classes=len(utils.NLCD_CLASSES), num_filters=64)